In [ ]:
import torch
#import lattice
from torch.utils.cpp_extension import load
lattice = load(name="lattice",sources=["lattice.cpp"])
import glob

In [ ]:
from crf.gaussian_matrix import LatticeGaussian#, LSHGaussian
from crf.crf import *
from crf.depth import *

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
def read_img(filename):
    img = Image.open(filename).convert('RGB')
    img = np.array(img).astype(float)/255
    return img

In [ ]:
paths = glob.glob('./images/*.bmp')
print(paths)

In [ ]:
img = read_img('./images/input.bmp')[::,::]
filtered_img = read_img('./images/bilateral_out_1.bmp')[::,::]
sigma_p = .01
sigma_c = .125

In [ ]:
h,w,c = img.shape
position = np.mgrid[:h,:w].transpose((1,2,0))/np.sqrt(h**2+w**2)
reference = np.zeros((h,w,5))
reference[...,:3] = img/sigma_c
reference[...,3:] = position/sigma_p
#reference = position/sigma_p
homo_src = np.ones((h,w,3+1))
homo_src[...,:c] = img

In [ ]:
ref_arr = torch.from_numpy(reference.reshape((h*w,-1)).astype(np.float32))
W = LatticeGaussian(ref_arr)
#W2 = gaussian_weights_u(ref_arr)#
src_arr = torch.from_numpy(homo_src.reshape((h*w,-1)).astype(np.float32))

In [ ]:
out = (W@src_arr+src_arr)#/(1+2**(-3))
a = out[:,:3]#/(out[:,-1][:,None])

In [ ]:
out2 = W2@src_arr+src_arr
b = out2[:,:3]#/(out2[:,-1][:,None])

In [ ]:
plt.hist(src_arr[:,:3].contiguous().view(-1).numpy(),alpha=.5)
plt.hist(a.contiguous().view(-1).numpy(),alpha=.5)
#plt.hist(b.contiguous().view(-1).numpy(),alpha=.5)
plt.legend(["input","lattice","brute_force"])

In [ ]:
#b = lattice.filter(src_arr,ref_arr)
#a = b[:,:3]#/(b[:,-1][:,None])

In [ ]:
f,axes = plt.subplots(1,3,figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')
axes[0].imshow(img)
#axes[1].imshow(b[:,:3].numpy().reshape((h,w,c)))
axes[2].imshow(a[:,:3].numpy().reshape((h,w,c)))
#axes[2].imshow(filtered_img)
plt.show()

In [ ]:
%timeit lattice.filter(src_arr,ref_arr)

In [1]:
%cd ~/crfrnn_layer/
import sys,os
import numpy as np
from scipy.misc import imread, imsave
import theano
import theano.tensor as tt
sys.path.append(os.path.expanduser('~/crfrnn_layer/'))
from crfrnn.gfilt import gaussian_filter

/home/marc/crfrnn_layer
using gpu


/home/marc/anaconda3/lib/python3.6/site-packages/theano/gpuarray/dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "


In [2]:
img = imread('input.bmp').astype(np.float32)[..., :3] / 255.
img = img.transpose(2,0,1)
yx = np.mgrid[:img.shape[1], :img.shape[2]].astype(np.float32)
stacked = np.vstack([yx, img])

kstd = np.array([5, 5, .125, .125, .125], np.float32)

R = tt.tensor3("R")
I = tt.tensor3("I")

N = gaussian_filter(R, tt.ones_like(I[:1, :, :]), kstd, 5, 1)
F = gaussian_filter(R, I, kstd, 5, 3) / N
bilateral = theano.function([R,I], F)

/home/marc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.


cpu gf?
cpu gf?


In [3]:
out = np.asarray(bilateral(stacked, img))

In [4]:
%timeit out = np.asarray(bilateral(stacked, img))

1.38 s ± 33.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
plt.imshow(out.transpose(1,2,0))

In [ ]:
np.asarray(bilateral(reference.astype(np.float32), img.astype(np.float32)))

In [ ]:
%run bilateral.py input.bmp output_.bmp 5 .125

In [ ]:
import pygpu

In [ ]:
pygpu.test()